In [97]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [99]:
link = r"C:\Users\User\Desktop\BD_tak\satellite.csv"

df = pd.read_csv(link)
df.head()

,position,status,name,model,operator,launch_place,launch_data
0,1 E,retired,BADR 3 (Arabsat 3A),Spacebus-3000B2,Arabsat,2700,26-Feb-1999
1,2 E,retired,Astra 1C,HS-601,SES S.A.,2790,12-May-1993
2,2 E,active,BulgariaSat-1,LS-1300,BulgariaSat,3669,23-Jun-2017
3,3 E,active,Eutelsat Konnect VHTS,Spacebus NEO,Eutelsat Communications S.A.,6396,7-Sep-2022
4,3 E,deorbited,Telecom 2C,Eurostar-2000,France Telecom,2283,6-Dec-1995


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   position      627 non-null    object
 1   status        627 non-null    object
 2   name          627 non-null    object
 3   model         627 non-null    object
 4   operator      627 non-null    object
 5   launch_place  627 non-null    object
 6   launch_data   627 non-null    object
dtypes: object(7)
memory usage: 34.4+ KB


In [103]:
df.describe()

,position,status,name,model,operator,launch_place,launch_data
count,627,627,627,627,627,627,627
unique,227,8,627,80,86,437,525
top,110 E,active,BADR 3 (Arabsat 3A),LS-1300,Intelsat,5500,1-May-2023
freq,14,362,1,120,83,9,3


In [105]:
for i in df.columns:
    print(f'Пропущенных значений в {i} равно {sum(df[i] == 'Nan')}')

Пропущенных значений в position равно 0
Пропущенных значений в status равно 0
Пропущенных значений в name равно 0
Пропущенных значений в model равно 0
Пропущенных значений в operator равно 0
Пропущенных значений в launch_place равно 7
Пропущенных значений в launch_data равно 0


In [107]:
df['launch_place'] = df['launch_place'].replace('Nan',0)

In [109]:
df.isna().sum()

position        0
status          0
name            0
model           0
operator        0
launch_place    0
launch_data     0
dtype: int64

In [113]:
df['launch_place'] = df['launch_place'].astype(int)

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   position      627 non-null    object
 1   status        627 non-null    object
 2   name          627 non-null    object
 3   model         627 non-null    object
 4   operator      627 non-null    object
 5   launch_place  627 non-null    int32 
 6   launch_data   627 non-null    object
dtypes: int32(1), object(6)
memory usage: 32.0+ KB


In [117]:
def convert_to_sql_date(df, date_column):

    df_result = df.copy()
    df_result[date_column] = (
        pd.to_datetime(
            df_result[date_column].astype(str).str.replace(r'^[^\d]*', '', regex=True),
            format='%d-%b-%Y',
            errors='coerce'
        )
        .dt.strftime('%Y-%m-%d')
        .replace('NaT', 'NULL')
    )
    
    return df_result

In [119]:
df_result = convert_to_sql_date(df, 'launch_data')
df_result.head()

,position,status,name,model,operator,launch_place,launch_data
0,1 E,retired,BADR 3 (Arabsat 3A),Spacebus-3000B2,Arabsat,2700,1999-02-26
1,2 E,retired,Astra 1C,HS-601,SES S.A.,2790,1993-05-12
2,2 E,active,BulgariaSat-1,LS-1300,BulgariaSat,3669,2017-06-23
3,3 E,active,Eutelsat Konnect VHTS,Spacebus NEO,Eutelsat Communications S.A.,6396,2022-09-07
4,3 E,deorbited,Telecom 2C,Eurostar-2000,France Telecom,2283,1995-12-06


In [121]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   position      627 non-null    object
 1   status        627 non-null    object
 2   name          627 non-null    object
 3   model         627 non-null    object
 4   operator      627 non-null    object
 5   launch_place  627 non-null    int32 
 6   launch_data   626 non-null    object
dtypes: int32(1), object(6)
memory usage: 32.0+ KB


In [127]:
launches_df = df_result[['position', 'status', 'launch_place', 'launch_data']].copy()


launches_df = launches_df.reset_index().rename(columns={'index': 'launch_id'})
launches_df['launch_id'] = launches_df['launch_id'] + 1  

objects_df = df_result[['name', 'model', 'operator']].copy()

objects_df = objects_df.reset_index().rename(columns={'index': 'launch_id'})
objects_df['launch_id'] = objects_df['launch_id'] + 1
objects_df['object_id'] = objects_df.index + 1  

In [129]:
launches_df.head()

,launch_id,position,status,launch_place,launch_data
0,1,1 E,retired,2700,1999-02-26
1,2,2 E,retired,2790,1993-05-12
2,3,2 E,active,3669,2017-06-23
3,4,3 E,active,6396,2022-09-07
4,5,3 E,deorbited,2283,1995-12-06


In [131]:
objects_df.head()

,launch_id,name,model,operator,object_id
0,1,BADR 3 (Arabsat 3A),Spacebus-3000B2,Arabsat,1
1,2,Astra 1C,HS-601,SES S.A.,2
2,3,BulgariaSat-1,LS-1300,BulgariaSat,3
3,4,Eutelsat Konnect VHTS,Spacebus NEO,Eutelsat Communications S.A.,4
4,5,Telecom 2C,Eurostar-2000,France Telecom,5
